In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime 
import time

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
tf.config.experimental.list_physical_devices(device_type = None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]

In [4]:
f16 = pd.read_csv('data/f16_l3.csv')

In [5]:
X = f16['Force']
y = f16['Acceleration3']

In [6]:
#X = MinMaxScaler().fit_transform(X.values.reshape(-1,1))

In [7]:
#Declare the number of model iterations we will run 
test_iterations = 250

#Declare the size of the initial train test split
test_size = 0.20

#Declare the number of hidden layers to add
layers = 2

#Declare the density of the hidden layers
density = 128

#Declare the type of activation for the hidden layers
activation = 'relu' 
#activation = ks.layers.LeakyReLU(alpha = 0.01)

#Assign the loss function the model will use to train
loss = 'mean_squared_error'

#Assign the optimizer for the compilitation
optimizer = 'adam'

#Declare the patience count used for early stopping
patience = 25

#Declare the test size used for early stopping train test split
es_test_size = 0.25

#Declare the batch size for use in the model
batch_size = 64

#Declare the maximum number of epochs for our model
epochs = 250

In [8]:
X_train, X_true, y_train, y_true = train_test_split(
    X, y, random_state = 8669, test_size = test_size)

In [9]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:0'):
    model = Sequential()

    #Creates a for loop that will add the number of layers based on the variable declared in the previous cell    
    for i in range(layers):
        #Adds a model layer with density and activation based on the variables declared in the previous cell
        model.add(Dense(density, activation = activation))
    
    model.add(Dense(1, activation='linear'))
    
    #model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

In [10]:
es = EarlyStopping(monitor = 'loss', patience = patience, restore_best_weights = True)
mc = ModelCheckpoint(filepath = 'test_model.h5', monitor = 'loss', save_best_only=True)
X_es_train, X_es_true, y_es_train, y_es_true = train_test_split(X_train, y_train, test_size = es_test_size, random_state = 8669)

In [11]:
#Creates a dataframe by which we will eventually put in our list created above
model_record = pd.DataFrame(columns = ['model_num', 'loss_type', 'time', 'epochs', 'r2', 'mae', 'mse', 'rmse', 'max_error'])

#Creates a dataframe by which our model's predicted values and true values will be stored
predict_record = pd.DataFrame(y_true).reset_index(drop = True)

#Creates a numpy array by which the for loop will use to count model runs and is then used to name df columns
model_counter = np.array([0])

In [12]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:0'):
    
#Performs model training repeatedly based on the variable declared previously    
    for i in range(test_iterations):
        model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

        #Creates an empty list for storing model metrics and other information 
        record_list = list() 
        #Starts a counter that adds 1 everytime a fitting is performed
        model_counter = model_counter + 1
        #Starts a timer to end after the round of fitting is complete
        start_time = datetime.now()
        
        #Fits our model/batch_size and epochs are declared previously
        #model.fit(x = X_train, y = y_train.values, 
        #          batch_size = batch_size, epochs = epochs)
        
        history = model.fit(x = X_es_train, y = y_es_train.values, 
                  validation_data = (X_es_true, y_es_true),
                  batch_size = batch_size, epochs = epochs, 
                  callbacks = [es, mc])
        
        #Saves our model predictions
        y_pred = model.predict(X_true)
        
        #Saves our model run #, time to run, and model metrics to our temporary list
        record_list.extend([len(model_record)+1, #A model iteration count
                            loss, #Records the loss metric used to fit the model
                            format(datetime.now() - start_time), #Calculates the time it took to complete the model
                            len(history.history['loss']), #The epoch number of our model checkpoint 
                            r2_score(y_true, y_pred), #R2
                            mean_absolute_error(y_true, y_pred), #MAE
                            mean_squared_error(y_true, y_pred), #MSE
                            np.sqrt(mean_squared_error(y_true, y_pred)), #RMSE
                            max_error(y_true, y_pred) #MaxError
                           ])        
        
        #Adds the temporary list of model metrics (etc) to the end of a dataframe
        model_record.loc[len(model_record)] = record_list
        
        #Converts our predictions to a dataframe so it will play nice
        y_pred_df = pd.DataFrame(y_pred)
        
        #Adds predictions as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, y_pred_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'm' + str(model_counter[0])})
        
        #Calculates the residual values for each prediction and stores it as a dataframe 
        residuals_df = pd.DataFrame(abs(predict_record.iloc[:,len(predict_record.columns)-1] - predict_record.iloc[:,0]))
        
        #Adds y residuals as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, residuals_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'res' + str(model_counter[0])})

Epoch 1/250
1006/1006 [==============================] - 4s 3ms/step - loss: 0.2564 - mse: 0.2564 - val_loss: 0.2408 - val_mse: 0.2408
Epoch 2/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2401 - mse: 0.2401 - val_loss: 0.2443 - val_mse: 0.2443
Epoch 3/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2364 - mse: 0.2364 - val_loss: 0.2403 - val_mse: 0.2403
Epoch 4/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2325 - mse: 0.2325 - val_loss: 0.2408 - val_mse: 0.2408
Epoch 5/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2306 - mse: 0.2306 - val_loss: 0.2394 - val_mse: 0.2394
Epoch 6/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2289 - mse: 0.2289 - val_loss: 0.2379 - val_mse: 0.2379
Epoch 7/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2278 - mse: 0.2278 - val_loss: 0.2370 - val_mse: 0.2370
Epoch 8/250
1006/1006 [==============================] 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2095 - mse: 0.2095 - val_loss: 0.2201 - val_mse: 0.2201
Epoch 62/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2100 - mse: 0.2100 - val_loss: 0.2174 - val_mse: 0.2174
Epoch 63/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2189 - val_mse: 0.2189
Epoch 64/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2098 - mse: 0.2098 - val_loss: 0.2235 - val_mse: 0.2235
Epoch 65/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2095 - mse: 0.2095 - val_loss: 0.2218 - val_mse: 0.2218
Epoch 66/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2095 - mse: 0.2095 - val_loss: 0.2192 - val_mse: 0.2192
Epoch 67/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2092 - mse: 0.2092 - val_loss: 0.2202 - val_mse: 0.2202
Epoch 68/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2177 - val_mse: 0.2177
Epoch 9/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2092 - mse: 0.2092 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 10/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2183 - val_mse: 0.2183
Epoch 11/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2094 - mse: 0.2094 - val_loss: 0.2190 - val_mse: 0.2190
Epoch 12/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2090 - mse: 0.2090 - val_loss: 0.2200 - val_mse: 0.2200
Epoch 13/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 14/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2184 - val_mse: 0.2184
Epoch 15/250
1006/1006 [==============================] - 3s 3

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2191 - val_mse: 0.2191
Epoch 69/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2180 - val_mse: 0.2180
Epoch 70/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2176 - val_mse: 0.2176
Epoch 71/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2182 - val_mse: 0.2182
Epoch 72/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2189 - val_mse: 0.2189
Epoch 73/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2177 - val_mse: 0.2177
Epoch 74/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 75/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 37/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2188 - val_mse: 0.2188
Epoch 38/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2081 - mse: 0.2081 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 39/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2179 - val_mse: 0.2179
Epoch 40/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 41/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 42/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2081 - mse: 0.2081 - val_loss: 0.2184 - val_mse: 0.2184
Epoch 43/250
1006/1006 [==============================] - 3s 

...

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 36/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 37/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 38/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 39/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2075 - mse: 0.2075 - val_loss: 0.2175 - val_mse: 0.2175
Epoch 40/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 41/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2166 - val_mse: 0.2166
Epoch 42/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 8/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 9/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 10/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 11/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 12/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 13/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 14/250
1006/1006 [==============================] - 3s 3m

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 3/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 4/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 5/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 6/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 7/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 8/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2179 - val_mse: 0.2179
Epoch 9/250
1006/1006 [==============================] - 3s 3ms/ste

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 30/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 31/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 32/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2175 - val_mse: 0.2175
Epoch 33/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2176 - val_mse: 0.2176
Epoch 34/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 35/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 36/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 32/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 33/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 34/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 35/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 36/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 37/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 38/250
1006/1006 [==============================] - 3s 

In [13]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:05:47.213460,113,0.192103,0.229841,0.215942,0.464696,5.215562
1,2,mean_squared_error,0:04:41.326302,92,0.173447,0.236953,0.220928,0.470030,5.221559
2,3,mean_squared_error,0:05:10.807415,103,0.189698,0.228515,0.216585,0.465387,5.225557
3,4,mean_squared_error,0:02:07.421104,42,0.191045,0.228586,0.216225,0.465000,5.219519
4,5,mean_squared_error,0:01:53.449180,37,0.188293,0.229163,0.216960,0.465790,5.234050
...,...,...,...,...,...,...,...,...,...
245,246,mean_squared_error,0:01:08.829938,26,0.194172,0.228406,0.215389,0.464100,5.224393
246,247,mean_squared_error,0:01:41.426324,39,0.194210,0.227621,0.215379,0.464089,5.231381
247,248,mean_squared_error,0:01:24.802321,33,0.187399,0.231580,0.217199,0.466047,5.188779
248,249,mean_squared_error,0:02:30.265147,58,0.188316,0.232261,0.216954,0.465783,5.218553


In [14]:
predict_record

,Acceleration3,m1,res1,m2,res2,m3,res3,m4,res4,m5,...,m246,res246,m247,res247,m248,res248,m249,res249,m250,res250
0,-0.314890,-0.091022,0.223868,-0.106568,0.208322,-0.108770,0.206120,-0.105270,0.209620,-0.119360,...,-0.102672,0.212218,-0.110335,0.204555,-0.075639,0.239251,-0.103132,0.211758,-0.108772,0.206118
1,-0.267150,-0.329607,0.062457,-0.462046,0.194896,-0.266110,0.001040,-0.289839,0.022689,-0.278545,...,-0.303950,0.036800,-0.294458,0.027308,-0.251692,0.015458,-0.257002,0.010148,-0.298067,0.030917
2,0.257710,0.322901,0.065191,0.297099,0.039389,0.270975,0.013265,0.256166,0.001544,0.297652,...,0.294082,0.036372,0.279458,0.021748,0.267252,0.009542,0.314668,0.056958,0.307198,0.049488
3,-0.193330,-0.089892,0.103438,-0.095056,0.098274,-0.099075,0.094255,-0.093161,0.100169,-0.108247,...,-0.099646,0.093684,-0.106840,0.086490,-0.066758,0.126572,-0.096786,0.096544,-0.105390,0.087940
4,-0.409560,-0.118341,0.291219,-0.128356,0.281204,-0.129487,0.280073,-0.126749,0.282811,-0.141855,...,-0.125896,0.283664,-0.133398,0.276162,-0.106510,0.303050,-0.120528,0.289032,-0.132094,0.277466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,-0.021237,-0.084471,0.063234,-0.090961,0.069724,-0.092564,0.071327,-0.082472,0.061235,-0.102835,...,-0.080187,0.058950,-0.099136,0.077899,-0.047220,0.025983,-0.075962,0.054725,-0.090486,0.069249
21447,-0.264320,0.083653,0.347973,0.076343,0.340663,0.084899,0.349219,0.092592,0.356912,0.083660,...,0.088426,0.352746,0.083829,0.348149,0.066600,0.330920,0.041781,0.306101,0.080519,0.344839
21448,0.060178,-0.093262,0.153440,-0.109899,0.170077,-0.111949,0.172127,-0.108569,0.168747,-0.122802,...,-0.103148,0.163326,-0.110888,0.171066,-0.077102,0.137280,-0.103509,0.163687,-0.109340,0.169518
21449,0.535320,0.295600,0.239720,0.270240,0.265080,0.246463,0.288857,0.231173,0.304147,0.249229,...,0.247994,0.287326,0.236267,0.299053,0.222682,0.312638,0.267925,0.267395,0.263516,0.271804


In [15]:
#Nifty call to confirm our variables were properly inputted into our model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               256       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 16,897
Trainable params: 16,897
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:05:47.213460,113,0.192103,0.229841,0.215942,0.464696,5.215562
1,2,mean_squared_error,0:04:41.326302,92,0.173447,0.236953,0.220928,0.470030,5.221559
2,3,mean_squared_error,0:05:10.807415,103,0.189698,0.228515,0.216585,0.465387,5.225557
3,4,mean_squared_error,0:02:07.421104,42,0.191045,0.228586,0.216225,0.465000,5.219519
4,5,mean_squared_error,0:01:53.449180,37,0.188293,0.229163,0.216960,0.465790,5.234050
...,...,...,...,...,...,...,...,...,...
245,246,mean_squared_error,0:01:08.829938,26,0.194172,0.228406,0.215389,0.464100,5.224393
246,247,mean_squared_error,0:01:41.426324,39,0.194210,0.227621,0.215379,0.464089,5.231381
247,248,mean_squared_error,0:01:24.802321,33,0.187399,0.231580,0.217199,0.466047,5.188779
248,249,mean_squared_error,0:02:30.265147,58,0.188316,0.232261,0.216954,0.465783,5.218553


In [17]:
model_record.to_csv('data/rnn_results_2_2.csv')
predict_record.to_csv('data/rnn_predictions_2_2.csv')